# Truncating an MPS

## Algorithm

The "simplification" algorithm approximates a MPS quantum state with a large bond dimension, $|\Psi\rangle,$ using an MPS with a smaller bond dimension $\xi$, $|\phi\rangle\in\mathrm{MPS}_\xi$. The optimal approximation is found by minimizing the distance between both states
$$\phi = \mathrm{argmin}_{\phi\in \mathrm{MPS}_\xi} d(\Psi,\phi).$$

The algorithm expresses this distance as a sum of scalar products
$$d(\Psi,\phi) = \|\Psi-\phi\|^2 = \langle\Psi|\Psi\rangle + \langle\phi|\phi\rangle - \langle\Psi|\phi\rangle - \langle\phi|\Psi\rangle,$$
all of which can be efficiently computed with MPS. Moreover, due to scalar product structure, the distance between two MPS's is a bilinear function with respect to any tensor in the unknown state $|\phi\rangle.$ This leads to a functional that can be efficiently optimized using an iterative algorithm.

### Variant 1: single site algorithm

Take for instance an MPS with four sites and let us focus on the second site. The scalar product between $\Psi$ and $\phi$ can be written as the contraction between two environments and two tensors from each state

<img src="figures/scalar-product-with-environments.svg" style="max-width:90%; width:30em">

As explained in [this notebook](File%201c%20-%20Canonical%20form.ipynb), if $|\phi\rangle$ is in canonical form with respect to that site., its norm can be written directly as the contraction of the same tensor and its complex conjugate.

<img src="figures/scalar-product-canonical.svg" style="max-width:90%; width:30em">

This structure allows us to write the condition for optimality with respect to a site
$$\frac{\partial}{\partial D^{i}_{\alpha\beta}} d(\Psi,\phi) = \frac{\partial}{\partial D^{i *}_{\alpha\beta}} d(\Psi,\phi) = 0$$
Since we have
$$\frac{\partial}{\partial D^{i *}_{\alpha\beta}} \langle\phi|\Psi\rangle = U^{i}_{\alpha\beta}$$
and
$$\frac{\partial}{\partial D^{i *}_{\alpha\beta}} \langle\phi|\phi\rangle = D^{i}_{\alpha\beta},$$
the solution to this equation is
$$U^{i}_{\alpha\beta} - D^{i}_{\alpha\beta} = 0,$$
where the tensor $U$ is constructed from the left environment $L,$ the right environment $R$ and the local tensor $B$ in the expression for $\langle{\phi|\Psi}\rangle.$

### Variant 2: two-site method

The algorithm in this form has a drawback: *we need to know the size of the tensor* $D^i_{\alpha\beta}$ in advance, because this size is never going to change throughout the algorithm. This is problematic: if we choose too small tensors, the approximation will be bad; if we choose big tensors, we will waste space and we will create states with zero Schmidt numbers, which will make many of our algorithms unstable and error prone.

A solution to this problem is to perform the same optimization but update two sites at the same time. In doing so, we will increase or reduce the bond dimension between both sites, adapting it to have good accuracy and numerical stability. Take for instance the example above and let us assume we want to optimize $D$ and $F$ simultaneously. We combine both sites into a single, larger tensor. This tensor satisfies also a linear equation. The two-site tensor is then split optimally, distributing the entanglement according to the maximum bond dimension that is allowed.

<img src="figures/two-site-optimization.svg" style="max-width:60%; width: 25em">

## Antilinear form

Both algorithms are implemented similarly, using the same abstraction. The key point is to realize that the solution of the problem
$$D^i_{\alpha\beta} = U^i_{\alpha\beta} = \frac{\partial}{\partial D^{i *}_{\alpha\beta}} \langle{\phi|\Psi}\rangle$$
is expressed in terms of an antiliear form that maps the local tensor $D^{i}_{\alpha\beta}$ of $|\phi\rangle$ to the scalar product $\langle{\phi|\Psi}\rangle.$

More precisely, if we define
$$\mathcal{L}[|\phi\rangle] := \langle\phi|\Psi\rangle,$$
this antilinear function of the tensor $D$ given by
$$\mathcal{L}[|\phi\rangle] = \sum_{i\alpha\beta} \sum U^{i}_{\alpha\beta} D^{i *}_{\alpha\beta}.$$

Similarly, there is a two-site tensor such that
$$\mathcal{L}[|\phi\rangle] = \sum_{ij\alpha\beta\gamma} U^{ij}_{\alpha\gamma} D^{i *}_{\alpha\beta}F^{j *}_{\beta\gamma}.$$

Our algorithm therefore revolves around creating an object type called `AntilinearForm` that takes two states $|\Psi\rangle$ and $|\phi\rangle,$ one of them in canonical form around the $n$-th site, and returns the tensor $C$ that represents the linear form with respect to that site.

As anticipated above, the core of the algorithm is to engineer the `AntilinearForm` class that keeps track of the environments and tensors---$L$, $C$ and $R$--with which we calculate the scalar product between our unknown $|\phi\rangle$ and the vector we wish to approximate $|\psi\rangle.$ This class will have two methods, `tensor1site()` and `tensor2site()` that will return the tensor $U$ or its extension to two sites.

It is important to remark that the functions construting the tensors, `tensor1site()` and `tensor2sites()`, are deeply tied to the implementation of environments in [notebook 2a](File%202a%20-%20Expectation%20values.ipynb). We copy here those conventions in graphical form

<img src="figures/linear-form-contraction-order.svg" style="max-width:90%; width:20em">

In [ ]:
# file: mps/truncate.py

import numpy as np
import math
import mps
import mps.state
from mps.tools import log
from mps.expectation import \
    begin_environment, \
    update_right_environment, \
    update_left_environment, \
    scprod

DEFAULT_TOLERANCE = np.finfo(np.float64).eps

class AntilinearForm:
    #
    # This class is an object that formally implements <ϕ|ψ> with an argument
    # ϕ that may change and be updated over time.
    #
    # Given a site 'n' it returns the tensor 'L' such that the contraction
    # between 'L' and ϕ[n] is the result of the linear form."""
    #
    def __init__(self, bra, ket, center=0):
        assert bra.size == ket.size
        #
        # At the beginning, we create the right- and left- environments for
        # all sites to the left and to the right of 'center', which is the
        # focus point of 'LinearForm'.
        #
        size = bra.size
        ρ = begin_environment()
        R = [ρ] * size
        for i in range(size-1, center, -1):
            R[i-1] = ρ = update_right_environment(bra[i], ket[i], ρ)

        ρ = begin_environment()
        L = [ρ] * size
        for i in range(0, center):
            L[i+1] = ρ = update_left_environment(bra[i], ket[i], ρ)

        self.bra = bra
        self.ket = ket
        self.size = size
        self.R = R
        self.L = L
        self.center = center

    def tensor1site(self):
        #
        # Return the tensor that represents the LinearForm at the 'center'
        # site of the MPS
        #
        center = self.center
        L = self.L[center]
        R = self.R[center]
        C = self.ket[center]
        return np.einsum('li,ijk,kn->ljn', L, C, R)

    def tensor2site(self, direction):
        #
        # Return the tensor that represents the LinearForm using 'center'
        # and 'center+/-1'
        #
        center = self.center
        if direction > 0:
            i = self.center
            j = i + 1
        else:
            j = self.center
            i = j - 1
        L = self.L[i]
        A = self.ket[i]
        B = self.ket[j]
        R = self.R[j]
        LA = np.einsum('li,ijk->ljk', L, A)
        BR = np.einsum('kmn,no->kmo', B, R)
        return np.einsum('ljk,kmo->ljmo', LA, BR)

    def update(self, direction):
        #
        # We have updated 'ϕ' (the bra), which is now centered on a different point.
        # We have to recompute the environments.
        #
        prev = self.center
        if direction > 0:
            nxt = prev + 1
            if nxt < self.size:
                self.L[nxt] = update_left_environment(self.bra[prev], self.ket[prev], self.L[prev])
                self.center = nxt
        else:
            nxt = prev-1
            if nxt >= 0:
                self.R[nxt] = update_right_environment(self.bra[prev], self.ket[prev], self.R[prev])
                self.center = nxt

## Two-site simplification loop

In [ ]:
# file: mps/truncate.py

def simplify(ψ, maxsweeps=4, direction=+1,
             tolerance=DEFAULT_TOLERANCE, normalize=True,
             max_bond_dimension=None):
    """Simplify an MPS ψ transforming it into another one with a smaller bond
    dimension, sweeping until convergence is achieved.
    
    Arguments:
    ----------
    ψ         -- state to approximate
    direction -- +1/-1 for the direction of the first sweep
    maxsweeps -- maximum number of sweeps to run
    tolerance -- relative tolerance when splitting the tensors
    max_bond_dimension -- maximum bond dimension (defaults to None, which is ignored)
    
    Output
    ------
    φ         -- CanonicalMPS approximation to the state ψ
    error     -- error made in the approximation, as $‖φ/‖φ‖ - ψ/‖ψ‖‖^2$
    direction -- direction that the next sweep would be
    """
    size = ψ.size
    start = 0 if direction > 0 else size-1

    base_error = ψ.error()
    φ = mps.state.CanonicalMPS(ψ, center=start, tolerance=tolerance, normalize=normalize)
    if max_bond_dimension == 0 and tolerance <= 0:
        return φ
    
    form = AntilinearForm(φ, ψ, center=start)
    norm_ψsqr = scprod(ψ, ψ).real
    err = 1.0
    log(f'Approximating ψ with |ψ|={norm_ψsqr**0.5} for {maxsweeps} sweeps.')
    for sweep in range(maxsweeps):
        if direction > 0:
            for n in range(0, size-1):
                log(f'Updating sites ({n},{n+1}) left-to-right, form.center={form.center}, φ.center={φ.center}')
                φ.update_2site(form.tensor2site(direction), n, direction, tolerance, normalize, max_bond_dimension)
                form.update(direction)
            last = size-1
        else:
            for n in reversed(range(0, size-1)):
                log(f'Updating sites ({n},{n+1}) right-to-left, form.center={form.center}, φ.center={φ.center}')
                φ.update_2site(form.tensor2site(direction), n, direction, tolerance, normalize, max_bond_dimension)
                form.update(direction)
            last = 0
        #
        # We estimate the error
        # 
        last = φ.center
        B = φ[last]
        norm_φsqr = np.vdot(B, B).real
        if normalize:
            φ[last] = B = B/norm_φsqr
            norm_φsqr = 1.
        scprod_φψ = np.vdot(B, form.tensor1site())
        old_err = err
        err = 2 * abs(1.0 - scprod_φψ.real/np.sqrt(norm_φsqr*norm_ψsqr))
        log(f'rel.err.={err}, old err.={old_err}, |φ|={norm_φsqr**0.5}')
        if err < tolerance:
            log(f'Stopping, as tolerance reached')
            break
        direction = -direction
    φ._error = 0.
    φ.update_error(base_error)
    φ.update_error(err)
    return φ, err, direction

The following function uses the simplification routine. We create a state $\Psi$ that is a product state and introduce random isometries between each pair of sites to arbitrarily increase the bond dimension. We then use `simplify()` to find a good approximation below a certain tolerance.

In [ ]:
def test(size=10, d=2, D=4):
    def rvector(d):
        v = np.random.rand(d)
        return v / np.linalg.norm(v)

    ψ = mps.state.MPS([rvector(d).reshape(1,d,1) for i in range(size)])
    print(f'|ψ|={ψ.norm2()}')
    
    for i in range(1,size):
        u = rvector(D)
        ψ[i-1] = ψ[i-1] * u.reshape(1,1,D)
        ψ[i] = ψ[i] * u.reshape(D,1,1)
    print(f'|ψ|={ψ.norm2()}')
    
    φ, err, _ = simplify(ψ, maxsweeps=4, normalize=True)
    print(f'|φ-ψ|={np.linalg.norm(φ.tovector()-ψ.tovector())}, err={np.sqrt(err)}')

test()

## Linear combinations

Using the simplification algorithms we can also solve the problem of finding the best MPS that approximates a linear combination of states, i.e.
$$\mathrm{argmin}_{\phi\in\mathrm{MPS}}\, \left\Vert \phi - \sum_{n=1}^N \alpha_n \Psi_n\right\Vert^2.$$

The algorithm for this is a trivial extension of the simplification algorithm. We have to build $N$ linear forms, one for each scalar product $\langle{\phi|\Psi_n}\rangle$ and find the local representations of those forms $U^{(n)},$ with which
$$D^i_{\alpha,\beta} = \sum_n \alpha_n U^{(n) i}_{\alpha,\beta}.$$

In [ ]:
# file: mps/truncate.py

def zero_mps(dimensions):
    return 

def multi_norm2(α, ψ):
    """Compute the norm-squared of the vector sum(α[i]*ψ[i])"""
    c = 0.
    for (i,αi) in enumerate(α):
        for j in range(i):
            c += 2*(αi.conjugate()*α[j]*scprod(ψ[i],ψ[j])).real
        c += np.abs(αi)**2 * scprod(ψ[i],ψ[i]).real
    return c

def combine(weights, states, guess=None, maxsweeps=4, direction=+1,
            tolerance=DEFAULT_TOLERANCE, normalize=True,
            max_bond_dimension=None):
    """Simplify an MPS ψ transforming it into another one with a smaller bond
    dimension, sweeping until convergence is achieved.
    
    Arguments:
    ----------
    weights   -- N values of α
    states    -- N MPS states
    guess     -- An MPS, defaults to states[0]
    direction -- +1/-1 for the direction of the first sweep
    maxsweeps -- maximum number of sweeps to run
    tolerance -- relative tolerance when splitting the tensors
    max_bond_dimension -- maximum bond dimension (defaults to None, which is ignored)
    
    Output
    ------
    φ         -- CanonicalMPS approximation to the linear combination state
    error     -- error made in the approximation
    """
    start = 0 if direction > 0 else size-1
    
    if guess is None:
        guess = states[0]
    base_error = sum(np.sqrt(np.abs(α))*np.sqrt(ψ.error()) for α,ψ in zip(weights, states))
    φ = mps.state.CanonicalMPS(guess, center=start, tolerance=tolerance)
    err = norm_ψsqr = multi_norm2(weights, states)
    if norm_ψsqr < tolerance:
        return mps.state.MPS([np.zeros((1,P.shape[1],1)) for P in φ]), 0
    log(f'Approximating ψ with |ψ|={norm_ψsqr**0.5} for {maxsweeps} sweeps.')
    log(f'Weights: {weights}')

    L = len(states)
    size = φ.size
    forms = [AntilinearForm(φ, ψ, center=start) for ψ in states]
    for sweep in range(maxsweeps):
        if direction > 0:
            for n in range(0, size-1):
                log(f'Updating sites ({n},{n+1}) left-to-right, form.center={forms[0].center}, φ.center={φ.center}')
                tensor = sum(α * f.tensor2site(direction) for α, f in zip(weights, forms))
                φ.update_2site(tensor, n, direction, tolerance, normalize, max_bond_dimension)
                for f in forms: f.update(direction)
        else:
            for n in reversed(range(0, size-1)):
                log(f'Updating sites ({n},{n+1}) right-to-left, form.center={forms[0].center}, φ.center={φ.center}')
                tensor = sum(α * f.tensor2site(direction) for α, f in zip(weights, forms))
                φ.update_2site(tensor, n, direction, tolerance, normalize, max_bond_dimension)
                for f in forms: f.update(direction)
            last = 0
        #
        # We estimate the error
        # 
        last = φ.center
        B = φ[last]
        norm_φsqr = np.vdot(B, B).real
        if normalize:
            φ[last] = B/norm_φsqr
            norm_φsqr = 1.
        C = sum(α * f.tensor1site() for α,f in zip(weights, forms))
        scprod_φψ = np.vdot(B, C)
        old_err = err
        err = 2 * abs(1.0 - scprod_φψ.real/np.sqrt(norm_φsqr*norm_ψsqr))
        log(f'rel.err.={err}, old err.={old_err}, |φ|={norm_φsqr**0.5}')
        if err < tolerance:
            log(f'Stopping, as tolerance reached')
            break
        direction = -direction
    φ._error = 0.
    φ.update_error(base_error**2)
    φ.update_error(err)
    return φ, err

The following function uses the simplification routine. We create a state $\Psi$ that is a product state and introduce random isometries between each pair of sites to arbitrarily increase the bond dimension. We then use `simplify()` to find a good approximation below a certain tolerance.

In [ ]:
def test_combine(size=10, d=2, D=4):
    def rvector(d):
        v = np.random.rand(d)
        return v / np.linalg.norm(v)

    # First random MPS
    ψ1 = mps.state.MPS([rvector(d).reshape(1,d,1) for i in range(size)])
    print(f'|ψ1|={ψ1.norm2()}')
    
    # Second random MPS
    ψ2 = mps.state.MPS([rvector(d).reshape(1,d,1) for i in range(size)])
    print(f'|ψ2|={ψ2.norm2()}')
    
    # Combine them
    αs = [1.0, -0.4]
    ψ = αs[0] * ψ1.tovector() + αs[1] * ψ2.tovector()
    print(f'|ψ|=|α1*ψ1+α2*ψ2|={np.linalg.norm(ψ)}')
    
    # Approximate 1.0 * ψ1 - 0.4 * ψ2
    φ, err = combine(αs, [ψ1, ψ2], maxsweeps=4, normalize=False)
    
    # Test the approximation
    print(f'|φ-ψ|={np.linalg.norm(φ.tovector()-ψ)}, err={np.sqrt(err)}')

test_combine()

In [ ]:
# file: mps/truncate.py

def cgs(A, b, guess=None, maxiter=100, tolerance=DEFAULT_TOLERANCE):
    """Given the MPO `A` and the MPS `b`, estimate another MPS that
    solves the linear system of equations A * ψ = b, using the
    conjugate gradient system.
    
    Parameters
    ----------
    A         -- Linear MPO
    b         -- Right-hand side of the equation
    maxiter   -- Maximum number of iterations
    tolerance -- Truncation tolerance and also error tolerance
    max_bond_dimension -- None (ignore) or maximum bond dimension
    
    Output
    ------
    ψ         -- Approximate solution to A ψ = b
    error     -- norm square of the residual, ||r||^2
    """
    ρprev = 0
    normb = scprod(b, b).real
    x = guess
    r = b
    if x is not None:
        r, err = combine([1.0, -1.0], [b, A.apply(x)],
                         tolerance=tolerance, normalize=False)
    p = r
    ρ = scprod(r, r).real
    for i in range(maxiter):
        Ap = A.apply(p)
        α = ρ / scprod(p, Ap).real
        if x is not None:
            x, err = combine([1, α], [x, p], tolerance=tolerance, normalize=False)
        else:
            x, err = combine([α], [p], tolerance=tolerance, normalize=False)
        r, err = combine([1, -1], [b, A.apply(x)], tolerance=tolerance, normalize=False)
        ρ, ρold = scprod(r, r).real, ρ
        ρ2 = multi_norm2([1.0, -1.0], [b, A.apply(x)])
        if ρ < tolerance * normb:
            log(f'Breaking on convergence')
            break
        p, err = combine([1., ρ/ρold], [r, p], tolerance=tolerance, normalize=False)
        log(f'Iteration {i:5}: |r|={ρ:5g}')
    return x, abs(ρ)

In [ ]:
from mps.mpo import MPO

def test(size=10, d=2, D=4):
    def rvector(d):
        v = np.random.rand(d)
        return v / np.linalg.norm(v)
    
    def flip_mpo(size):
        A = np.zeros((1,2,2,1))
        A[0,0,1,0] = 1.0
        A[0,1,0,0] = 1.0
        return MPO([A]*size)

    ψ = mps.state.MPS([rvector(d).reshape(1,d,1) for i in range(size)])
    print(f'|ψ|={ψ.norm2()}')
    
    A = flip_mpo(size)
    φ, err = cgs(A, ψ, guess=ψ, maxiter=10)
    Aφ = A.apply(φ)
    print(f'|φ-ψ|={np.linalg.norm(Aφ.tovector()-ψ.tovector())}, err={np.sqrt(err)}')

test()

---

# Tests

In [ ]:
# file: mps/test/test_truncate.py

import numpy as np
import unittest
from mps.test.tools import *
from mps.state import CanonicalMPS, MPS
from mps.truncate import simplify, combine, AntilinearForm
from mps.expectation import expectation1, expectation2

class TestLinearForm(unittest.TestCase):
    
    def test_canonical_env(self):
        #
        # When we pass two identical canonical form MPS to LinearForm, the
        # left and right environments are the identity
        #
        def ok(ψ):
            global foo
            for center in range(ψ.size):
                ϕ = CanonicalMPS(ψ, center=center, normalize=True)
                LF = AntilinearForm(ϕ, ϕ, center)
                for i in range(ϕ.size):
                    if i <= center:
                        self.assertTrue(similar(LF.L[i], ϕ.left_environment(i)))
                        self.assertTrue(almostIdentity(LF.L[i],+1))
                    if i >= center:
                        self.assertTrue(similar(LF.R[i], ϕ.right_environment(i)))
                        self.assertTrue(almostIdentity(LF.R[i],+1))
        
        run_over_random_mps(ok)
    
    def tensor1siteok(self, aϕ, O):
        for center in range(aϕ.size):
            ϕ = CanonicalMPS(aϕ, center=center, normalize=True)
            for n in range(ϕ.size):
                #
                # Take an MPS Φ, construct a new state ψ = O1*ϕ with a local
                # operator on the 'n-th' site
                #
                ψ = mps.state.MPS(ϕ)
                ψ[n] = np.einsum('ij,ajb->aib', O, ψ[n])
                #
                # and make sure that the AntilinearForm provides the right tensor to
                # compute <ϕ|ψ> = <ϕ|O1|ϕ>
                #
                Odesired = expectation1(ϕ, O, n)
                LF = AntilinearForm(ϕ, ψ, center)
                Oestimate = np.einsum('aib,aib', ϕ[center].conj(), LF.tensor1site())
                self.assertAlmostEqual(Oestimate, Odesired)
                if n >= center:
                    self.assertTrue(almostIdentity(LF.L[center],+1))
                if n <= center:
                    self.assertTrue(almostIdentity(LF.R[center],+1))

    def test_tensor1site_product(self):
        O = np.array([[0.3,0.2+1.0j],[0.2-1.0j,2.0]])
        run_over_random_mps(lambda ϕ: self.tensor1siteok(ϕ, O), D=1)

    def test_tensor1site(self):
        O = np.array([[0.3,0.2+1.0j],[0.2-1.0j,2.0]])
        run_over_random_mps(lambda ϕ: self.tensor1siteok(ϕ, O))
    
    def tensor2siteok(self, aϕ, O1, O2):
        for center in range(aϕ.size):
            ϕ = CanonicalMPS(aϕ, center=center, normalize=True)
            for n in range(ϕ.size-1):
                #
                # Take an MPS Φ, construct a new state ψ = O1*ϕ with a local
                # operator on the 'n-th' site
                #
                ψ = mps.state.MPS(ϕ)
                ψ[n] = np.einsum('ij,ajb->aib', O1, ψ[n])
                ψ[n+1] = np.einsum('ij,ajb->aib', O2, ψ[n+1])
                #
                # and make sure that the AntilinearForm provides the right tensor to
                # compute <ϕ|ψ> = <ϕ|O1|ϕ>
                #
                Odesired = ϕ.expectation2(O1, O2, n)
                LF = AntilinearForm(ϕ, ψ, center)
                if center+1 < ϕ.size:
                    D = np.einsum('ijk,klm->ijlm', ϕ[center], ϕ[center+1])
                    Oestimate = np.einsum('aijb,aijb', D.conj(), LF.tensor2site(+1))
                    self.assertAlmostEqual(Oestimate, Odesired)
                if center > 0:
                    D = np.einsum('ijk,klm->ijlm', ϕ[center-1], ϕ[center])
                    Oestimate = np.einsum('aijb,aijb', D.conj(), LF.tensor2site(-1))
                    self.assertAlmostEqual(Oestimate, Odesired)
                if n >= center:
                    self.assertTrue(almostIdentity(LF.L[center],+1))
                if n+1 <= center:
                    self.assertTrue(almostIdentity(LF.R[center],+1))

    def test_tensor2site_product(self):
        O1 = np.array([[0.3,0.2+1.0j],[0.2-1.0j,2.0]])
        O2 = np.array([[0.34,0.4-0.7j],[0.4+0.7j,-0.6]])
        run_over_random_mps(lambda ϕ: self.tensor2siteok(ϕ, O1, O2), D=1)

    def test_tensor2site(self):
        O1 = np.array([[0.3,0.2+1.0j],[0.2-1.0j,2.0]])
        O2 = np.array([[0.34,0.4-0.7j],[0.4+0.7j,-0.6]])
        run_over_random_mps(lambda ϕ: self.tensor2siteok(ϕ, O1, O2))

In [ ]:
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestLinearForm'])
unittest.TextTestRunner(verbosity=2).run(suite1);